# 1_combine.ipynb  
### Team: Team Yunus
### Made by: Yunus Eren Ertas

This notebook loads all the raw NESO electricity demand CSV files, detects the 
correct timestamp and demand columns for each year, and combines everything 
into one unified dataset.

Because NESO changes file formats frequently (column names, datetime formats, 
etc.), a robust loader is needed. The output of this notebook is a single 
`uk_electricity_combined.csv` file that will be used in the cleaning stage.

### What this cell does

- Imports the required Python libraries.
- Sets the path to the folder where all raw NESO CSV files are stored.
- Lists all files that will be processed.
- This helps us confirm that all expected yearly files (2001–2025) are present.


In [1]:
import pandas as pd
from pathlib import Path

root = Path("../data/raw/electricity")
files = sorted(root.glob("*.csv"))

print("Number of files:", len(files))
for f in files:
    print("-", f.name)


Number of files: 25
- demanddata_2001.csv
- demanddata_2002.csv
- demanddata_2003.csv
- demanddata_2004.csv
- demanddata_2005.csv
- demanddata_2006.csv
- demanddata_2007.csv
- demanddata_2008.csv
- demanddata_2009.csv
- demanddata_2010.csv
- demanddata_2011.csv
- demanddata_2012.csv
- demanddata_2013.csv
- demanddata_2014.csv
- demanddata_2015.csv
- demanddata_2016.csv
- demanddata_2017.csv
- demanddata_2018.csv
- demanddata_2019.csv
- demanddata_2020.csv
- demanddata_2021.csv
- demanddata_2022.csv
- demanddata_2023.csv
- demanddata_2024.csv
- demanddata_2025.csv


### Define a robust CSV loader for NESO files

NESO changes column names and timestamp formats across years.  
This function:

- Normalizes column names  
- Searches multiple possible timestamp columns  
- Searches multiple possible demand columns (priority order)  
- Converts both fields to correct types  
- Returns only `timestamp` and `demand_mw`  

If a file lacks valid timestamp/demand columns, it is skipped.


In [2]:
def load_neso_csv(path):
    df = pd.read_csv(path)

    # Normalize column names
    df.columns = [c.lower().strip().replace(" ", "_") for c in df.columns]

    # --------------------
    # CASE 1: Settlement period data (old NESO format)
    # --------------------
    if "settlement_date" in df.columns and "settlement_period" in df.columns:
        
        df["settlement_date"] = pd.to_datetime(df["settlement_date"], errors="coerce")

        # Construct timestamp:
        # period 1 = 00:00
        # period 2 = 00:30
        # ...
        df["timestamp"] = df["settlement_date"] + pd.to_timedelta(
            (df["settlement_period"] - 1) * 30, unit="m"
        )

    # --------------------
    # CASE 2: Already has a timestamp column (newer NESO format)
    # --------------------
    else:
        time_candidates = ["datetime", "local_time", "time", "timestamp"]
        tcol = None
        for c in time_candidates:
            match = [x for x in df.columns if c in x]
            if match:
                tcol = match[0]
                break

        if tcol is None:
            print(f"⚠ No usable timestamp in {path.name}, skipping.")
            return None

        df["timestamp"] = pd.to_datetime(df[tcol], errors="coerce")

    # --------------------
    # Demand column detection
    # --------------------
    demand_priority = [
        "england_wales_demand",  # best
        "actual_demand",
        "nd",
        "tsd",
        "demand"
    ]

    dcol = None
    for k in demand_priority:
        match = [c for c in df.columns if k in c]
        if match:
            dcol = match[0]
            break

    if dcol is None:
        print(f"⚠ No demand in {path.name}, skipping.")
        return None

    df[dcol] = pd.to_numeric(df[dcol], errors="coerce")

    out = df[["timestamp", dcol]].dropna()
    out.columns = ["timestamp", "demand_mw"]

    return out


### Load all files using the robust loader

Each file is processed individually.  
If valid timestamp and demand columns are found, the file is added to `frames`.  
Otherwise, the file is skipped safely.


In [3]:
frames = []

for f in files:
    print("Reading:", f.name)
    df_part = load_neso_csv(f)
    if df_part is not None:
        frames.append(df_part)

Reading: demanddata_2001.csv
Reading: demanddata_2002.csv
Reading: demanddata_2003.csv
Reading: demanddata_2004.csv
Reading: demanddata_2005.csv
Reading: demanddata_2006.csv
Reading: demanddata_2007.csv
Reading: demanddata_2008.csv
Reading: demanddata_2009.csv


C:\Users\erene\AppData\Local\Temp\ipykernel_4840\1919508322.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["settlement_date"] = pd.to_datetime(df["settlement_date"], errors="coerce")
C:\Users\erene\AppData\Local\Temp\ipykernel_4840\1919508322.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["settlement_date"] = pd.to_datetime(df["settlement_date"], errors="coerce")
C:\Users\erene\AppData\Local\Temp\ipykernel_4840\1919508322.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["settlement_date"] = pd.to_datetime(df["settlement_date"], errors="coerce")
C:\U

Reading: demanddata_2010.csv
Reading: demanddata_2011.csv
Reading: demanddata_2012.csv
Reading: demanddata_2013.csv
Reading: demanddata_2014.csv
Reading: demanddata_2015.csv
Reading: demanddata_2016.csv


C:\Users\erene\AppData\Local\Temp\ipykernel_4840\1919508322.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["settlement_date"] = pd.to_datetime(df["settlement_date"], errors="coerce")
C:\Users\erene\AppData\Local\Temp\ipykernel_4840\1919508322.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["settlement_date"] = pd.to_datetime(df["settlement_date"], errors="coerce")
C:\Users\erene\AppData\Local\Temp\ipykernel_4840\1919508322.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["settlement_date"] = pd.to_datetime(df["settlement_date"], errors="coerce")
C:\U

Reading: demanddata_2017.csv
Reading: demanddata_2018.csv
Reading: demanddata_2019.csv
Reading: demanddata_2020.csv
Reading: demanddata_2021.csv
Reading: demanddata_2022.csv
Reading: demanddata_2023.csv
Reading: demanddata_2024.csv
Reading: demanddata_2025.csv


C:\Users\erene\AppData\Local\Temp\ipykernel_4840\1919508322.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["settlement_date"] = pd.to_datetime(df["settlement_date"], errors="coerce")
C:\Users\erene\AppData\Local\Temp\ipykernel_4840\1919508322.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["settlement_date"] = pd.to_datetime(df["settlement_date"], errors="coerce")


### Combine all valid partial DataFrames into one dataset

- Concatenates all data into a single DataFrame  
- Removes remaining invalid rows  
- Sorts chronologically  
- Displays the first rows for verification  


In [4]:

df = pd.concat(frames, ignore_index=True)
df = df.dropna().sort_values("timestamp").reset_index(drop=True)

df.head()

,timestamp,demand_mw
0,2001-01-01 00:00:00,34060
1,2001-01-01 00:30:00,35370
2,2001-01-01 01:00:00,35680
3,2001-01-01 01:30:00,35029
4,2001-01-01 02:00:00,34047


### Why this check is important

Each year should have either:
- **17,520 rows** (365 days × 48 half-hours)
- **17,568 rows** (leap year = 366 days × 48 half-hours)
- **Partial number** only if the year is not complete (e.g., 2025)

This confirms:
- All years loaded correctly.
- No missing years.
- No corrupted timestamp parsing.


In [5]:
df["year"] = df["timestamp"].dt.year
df["year"].value_counts().sort_index()


year
2001    17520
2002    17520
2003    17520
2004    17568
2005    17520
2006    17520
2007    17520
2008    17568
2009    17520
2010    17520
2011    17520
2012    17568
2013    17520
2014    17520
2015    17520
2016    17568
2017    17520
2018    17520
2019    17520
2020    17568
2021    17520
2022    17520
2023    17520
2024    17568
2025    14640
Name: count, dtype: int64

### Final step

We save the combined dataset in a clean 2-column CSV:
- `timestamp`
- `demand_mw`

This file will be used in the next notebook (`02-clean.ipynb`) where we will:
- Resample to hourly
- Handle missing values
- Remove invalid spikes
- Export the final cleaned dataset


In [6]:
out_path = Path("../data/raw/uk_electricity_combined.csv")
df[["timestamp", "demand_mw"]].to_csv(out_path, index=False)

print("Saved combined file to:", out_path)


Saved combined file to: ..\data\raw\uk_electricity_combined.csv
